<a href="https://colab.research.google.com/github/Jeong-So/06.08-07.02_Digital-Finance-Developer-Course-Using-AI/blob/main/(20220624)%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90_%EB%89%B4%EC%8A%A4_%EC%A3%BC%EC%8B%9D_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 삼성전자 주가 크롤링 및 뉴스 크롤링 데이터 분석

In [ ]:
# MeCab 설치
# 일반적인 한글 토큰화에서는 MeCab 사용 (transformer계열의 딥러닝은 다른 토큰화 사용)
!sudo apt-get install g++ openjdk-7-jdk # Install Java 1.7+
!sudo apt-get install python3-dev; pip3 install konlpy   # Python 3.x
!sudo apt-get install curl
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

# pyLDAvis 설치
# 토픽모델링에 필요한 pyLDAvis 사용
!pip install pyLDAvis

# punkt 다운로드
# 토큰화 함수 다운로드
import nltk
nltk.download('punkt')

# 한글폰트 설치
import matplotlib.pyplot as plt
import matplotlib as mpl 
import matplotlib.font_manager as fm 
!apt-get update -qq
!apt-get install fonts-nanum* -qq

path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'  # 설치된 나눔글꼴중 원하는 녀석의 전체 경로를 가져오자
font_name = fm.FontProperties(fname=path, size=15).get_name()
plt.rc('font', family=font_name)
fm._rebuild()
mpl.rcParams['axes.unicode_minus'] = False


# 1.1 시장지표 조회

In [32]:
import requests
import pandas as pd

code = '005390'
start = '20200101'
end = '20220624'

def get_stock_price(code, start, end) : 
  url = 'https://api.finance.naver.com/siseJson.naver?symbol={}&requestType=1&startTime={}&endTime={}&timeframe=day'.format(code, start, end)

  res = requests.post(url)
  tmp = res.text.replace('\n','').replace('\t','').replace(' ','').replace('"','').replace(']]','').split('],[')[1:]
  tmp2 = [item.split(',') for item in tmp]

  df = pd.DataFrame(columns=['date','open','high','low','close','volume'])

  for item in tmp2 : 
    df.loc[len(df)] = item[:-1] 

  df['date'] = pd.to_datetime(df['date'])
  df['open'] = df['open'].astype(float)
  df['high'] = df['high'].astype(float)
  df['low'] = df['low'].astype(float)
  df['close'] = df['close'].astype(float)
  df['volume'] = df['volume'].astype(float)
  return df.set_index('date')

df_price = get_stock_price(code, start, end)  # 삼성전자 항공 주가


In [4]:
import pandas as pd
import requests
from datetime import datetime
import xml.etree.ElementTree as ET  # xml.etree.ElementTree 활용하여 xlm 데이터 파싱해서 데이터 가져옴
from tqdm.notebook import tqdm as tqdm

# 주가 조회 함수 (주가 차트에서 조회)
def get_stock_price(code, startdate, enddate) :
  cnt = 2000    # 현재를 기준으로 몇개의 데이터를 가져오겠다 
  df = pd.DataFrame(columns=['date', 'open', 'high', 'low', 'close','volume'])

  url = "https://fchart.stock.naver.com/sise.nhn?symbol={}&timeframe=day&count={}&requestType=0".format(code, cnt) 
  # 위 url에서 데이터를 가져오면 포멧이 xml형태
  response = requests.get(url)
  e = ET.fromstring(response.text.replace('\n','').replace('\t',''))

  stock_history = []
  for item in tqdm(e.find('chartdata').findall('item'), desc="get_stock_price") :
    data = item.get('data').split('|')
    date = datetime.strptime(data[0],'%Y%m%d')
    if (datetime.strptime(enddate,'%Y%m%d') < date) :
      break
    elif (datetime.strptime(startdate,'%Y%m%d') <= date) & (datetime.strptime(enddate,'%Y%m%d') >= date) :
      df.loc[len(df)] = data

  df['date'] = pd.to_datetime(df['date'])
  #df['open'] = df['open'].astype(float)
  #df['high'] = df['high'].astype(float)
  df['close'] = df['close'].astype(float)
  #df['low'] = df['low'].astype(float)
  df['volume'] = df['volume'].astype(float)

  return df.set_index('date')

get_stock_price('020560', '20220201', '20220624')  

get_stock_price:   0%|          | 0/2000 [00:00<?, ?it/s]

,open,high,low,close,volume
date,,,,,
2022-02-03,17800,18350,17600,17850.0,391191.0
2022-02-04,17850,20250,17700,19750.0,1170538.0
2022-02-07,19750,20350,19250,20250.0,1527484.0
2022-02-08,20450,21000,20100,20750.0,1409973.0
2022-02-09,21000,21350,20750,21000.0,1124104.0
...,...,...,...,...,...
2022-06-20,16800,16950,15600,15750.0,559566.0
2022-06-21,15850,16400,15500,16400.0,302419.0
2022-06-22,16500,16550,15350,15500.0,433725.0


# 2. 전처리

##2.1 정제

In [9]:
import re
from nltk.tokenize import sent_tokenize
from tqdm.notebook import tqdm as tqdm

def cleansing(df) :
  for i in tqdm(range(df.shape[0]), desc="cleansing") : 
    cont = df['content'][i]
    try : 
      cont = ' '.join(sent_tokenize(cont)[:-1])
      cont = re.sub("(\..{0,20}\S+@\S+ 기사제보 및 보도자료.*무단전재 및 재배포 금지)", ".", cont)
      cont = re.sub("(\S+@\S+.*무단 전재-재배포 금지)", "", cont)
      cont = re.sub("(▶.{0,20}무단전재 및 재배포 금지)", "", cont)
      cont = re.sub("(\[.*무단전재 및 재배포 금지\])", "", cont)
      cont = re.sub("(\\S+@\S+ <.*무단전재-재배포 금지\.\>)", "", cont)
      cont = re.sub("(\..{0,10}\S+@\S+.*재배포 금지＞)", ".", cont)
      cont = re.sub("(\(.{0,20}\=.{0,20}기자\))", "", cont)
      cont = re.sub("(\..{0,20}기자\(.*\))", ".", cont)
      cont = re.sub("(\..{0,20}기자 \S+@\S+)", ".", cont)
      cont = re.sub("(【\S+=\S+】.{0,20}기자 =)", "", cont)
      cont = re.sub("(\[.{0,20}기자.{0,5}\])", "", cont)
      cont = re.sub("(\[\S+ \S+\])", "", cont)
      df['content'][i] = cont
    except :
      pass
  return df

##2.2 토큰화 / 품사태깅 / 불용어처리

In [10]:
from konlpy.tag import Mecab
from nltk.util import everygrams
from tqdm.notebook import tqdm as tqdm

def tokenize(df, selected_pos, stopwords) :
  m = Mecab()
  df['tokens'] = ''
  df['ngrams'] = ''
  for i, row in tqdm(df.iterrows(), total=df.shape[0], desc="tokenize"): 
    ngrams = []
    tokens = []
    li = []
    try : 
      for p in m.pos(row['content']) :
        if (p[1] in selected_pos) & (p[0] not in stopwords) :
          li.append(p[0])
          tokens.append(p[0])
        elif len(li) > 0 :
          ngrams = ngrams + [';'.join(ngrams) for ngrams in everygrams(li)]
          li=[]
    except Exception as e:
      pass
    df['tokens'][i] = ','.join(tokens)
    df['ngrams'][i] = ','.join(ngrams)

  return df

* 현재는 전처리 된 데이터 가져와서 처리

In [6]:
# 구글드라이브와 코랩 연동

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
# 구글드라이브 경로 설정 /content/드라이브 경로/파일

path = "/content/drive/MyDrive/Data/2_preprocessed.csv"
import pandas as pd
df = pd.read_csv(path, parse_dates=['date']).dropna()

In [16]:
df

,category1,category2,date,media,title,url,tokens,ngrams
0,경제,중기/벤처,2019-01-01,매일경제,40여국에 수출하는 한국뉴매틱 진공펌프,https://news.naver.com/main/read.nhn?mode=LS2D...,"車,제조,현장,필수,장비,국산화,제품,특허,해외,공략,가속,한국,뉴매틱,공장,자동,...","車,제조,현장,필수,장비,국산화,車;제조,제조;현장,현장;필수,필수;장비,장비;국산..."
1,경제,증권,2019-01-01,아시아경제,긴장 넘쳤던 2018 증시…코스피 2년 만에 2100 하회,https://news.naver.com/main/read.nhn?mode=LS2D...,"지난해,국내,증시,긴장감,코스피,지수,대,대,내림,반복,코스닥,대,대,일,한국,거래...","지난해,국내,증시,국내;증시,긴장감,코스피,지수,코스피;지수,대,대,내림,반복,코스..."
2,경제,증권,2019-01-01,매일경제,"SK 주요 관계사 경영진, CES 2019 참가…성장동력 찾기 적극 나선다",https://news.naver.com/main/read.nhn?mode=LS2D...,"주요,관계사,오,일,일,미국,라스베가스,열리,세계,최대,가전,전시회,참가,새해,성장...","주요,관계사,주요;관계사,오,일,일,미국,라스베가스,미국;라스베가스,열리,세계,최대..."
3,경제,증권,2019-01-01,한국경제TV,"[신년대기획] 투자, 판이 바뀐다 1부 위기의 파고를 넘어",https://news.naver.com/main/read.nhn?mode=LS2D...,"동영상,뉴스,대한민국,정부,수립,오늘,이르,한강,기적,이,만큼,경제,발전,배경,금융...","동영상,뉴스,대한민국,정부,수립,동영상;뉴스,뉴스;대한민국,대한민국;정부,정부;수립..."
4,경제,증권,2019-01-01,조선비즈,증권사 리서치센터장이 뽑은 새해 유망주식 5개,https://news.naver.com/main/read.nhn?mode=LS2D...,"년,황금,돼지해,바라보,증권사,리서치,센터,장,마음,무겁,그간,홀로,호황,미국,증시...","년,황금,돼지해,년;황금,황금;돼지해,년;황금;돼지해,바라보,증권사,리서치,센터,장..."
...,...,...,...,...,...,...,...,...
133250,생활/문화,생활문화 일반,2020-04-24,일간스포츠,"플친폰 “갤럭시S20 10만원대 특판...갤럭시노트10, S10 5G 할인”",https://news.naver.com/main/read.nhn?mode=LS2D...,"삼성전자,달,출시,상반기,전략,스마트폰,갤럭시,시리즈,코로나,확산,예상,밖,부진,겪...","삼성전자,달,출시,달;출시,상반기,전략,스마트폰,갤럭시,상반기;전략,전략;스마트폰,..."
133251,생활/문화,자동차/시승기,2020-04-24,동아일보,車-반도체 1분기 선방했지만… “코로나 쇼크 이제 시작”,https://news.naver.com/main/read.nhn?mode=LS2D...,"현대차,분기,영업,익,증가,삼성,반도체,작년,수준,조,예상,이달,들,반도체,수출,줄...","현대차,분기,영업,익,분기;영업,영업;익,분기;영업;익,증가,삼성,반도체,작년,수준..."
133252,생활/문화,건강정보,2020-04-24,서울경제,코로나 계기 20년만에 '물꼬'···선원 등 원격의료 허용할 듯,https://news.naver.com/main/read.nhn?mode=LS2D...,"서울,경제,한국,의사,환자,간,원격,의료,시범,사업,처음,진행,년,그,뒤,년,간,국...","서울,경제,서울;경제,한국,의사,환자,간,원격,의료,시범,사업,환자;간,간;원격,원..."
133253,정치,정치일반,2020-04-24,미디어오늘,민언련 악성 바이러스 전원책 발언 심의 접수,https://news.naver.com/main/read.nhn?mode=LS2D...,"전원책,변호사,유튜브,콘텐츠,출연,민주,언론,시민,연합,악성,바이러스,비난,데,민주...","전원책,변호사,전원책;변호사,유튜브,콘텐츠,유튜브;콘텐츠,출연,민주,언론,시민,연합..."


##3. 토픽모델링

##3.1 토픽모델링 LDA

In [ ]:
# 실습. 토픽모델링
# 데이터 : 2_preprocessed.csv 파일내 tokens 컬럼 활용
# 모델 : gensim LDA 모델 활용
# 토픽갯수 : 50개

In [17]:
from gensim import corpora
from gensim.models import LdaModel
from gensim.models import TfidfModel

preprocessed_docs = []

for i, row in df.iterrows():
  preprocessed_docs.append([w for w in row['tokens'].split(',') if len(w) > 1])
# float은 split 할 수 없어서 df가져올때 dropna 해줘야함

print(preprocessed_docs)

id2word = corpora.Dictionary(preprocessed_docs)
corpus = [id2word.doc2bow(doc) for doc in preprocessed_docs]
tfidf = TfidfModel(corpus)
corpus_TFIDF = tfidf[corpus]

model = LdaModel(corpus,id2word=id2word, num_topics= 50)



IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [26]:
# 토픽 출력

for topic in model.print_topics(num_topics=50, num_words=10):
  print(topic)

(0, '0.054*"기업" + 0.052*"그룹" + 0.040*"준법" + 0.028*"계열사" + 0.024*"현대차" + 0.022*"현대" + 0.022*"한진" + 0.018*"자동차" + 0.017*"삼성" + 0.015*"래미안"')
(1, '0.349*"코로나" + 0.042*"전자" + 0.027*"가전" + 0.023*"건조기" + 0.021*"세탁기" + 0.019*"제품" + 0.015*"효율" + 0.015*"에어컨" + 0.014*"에너지" + 0.011*"공기"')
(2, '0.144*"연봉" + 0.120*"평균" + 0.071*"보수" + 0.055*"목표" + 0.033*"의견" + 0.023*"유지" + 0.021*"제시" + 0.012*"성과급" + 0.012*"포토" + 0.012*"매수"')
(3, '0.030*"지수" + 0.029*"외국인" + 0.028*"코스피" + 0.025*"개인" + 0.023*"매수" + 0.021*"시장" + 0.021*"포인트" + 0.020*"하락" + 0.018*"상승" + 0.016*"코스닥"')
(4, '0.029*"플러스" + 0.028*"방역" + 0.026*"텔레콤" + 0.023*"서비스" + 0.022*"통신" + 0.019*"가입자" + 0.017*"게임" + 0.014*"이동" + 0.013*"지원금" + 0.012*"요금제"')
(5, '0.048*"무선" + 0.032*"워치" + 0.031*"혈압" + 0.027*"이어폰" + 0.025*"측정" + 0.022*"캠페인" + 0.022*"스마트" + 0.020*"배터리" + 0.019*"충전" + 0.017*"기기"')
(6, '0.146*"평가" + 0.129*"가치" + 0.061*"벨벳" + 0.020*"포브스" + 0.019*"데시" + 0.018*"분석" + 0.017*"빛나" + 0.016*"종합" + 0.014*"당기" + 0.013*"파워"')
(7, '0.027*"재판" + 0.027*"부회장" 

In [ ]:
# 문서별 토필 배정

In [30]:
df['topic'] = 0
df['topic_detail'] = 0.0

for i, row in df.iterrows():
  preprocessed_docs = [w for w in row['tokens'].split(',') if len(w) > 1 ]

  corpus = id2word.doc2bow(preprocessed_docs)
  topic_info = model[corpus]
  dominant_topic = sorted(topic_info, key=lambda x:x[1], reverse=True)[0]

  df.loc[i, 'topic'] = dominant_topic[0]
  df.loc[i, 'topic_details'] = dominant_topic[1]

In [31]:
df[:10]

,category1,category2,date,media,title,url,tokens,ngrams,topic,topic_detail,topic_details
0,경제,중기/벤처,2019-01-01,매일경제,40여국에 수출하는 한국뉴매틱 진공펌프,https://news.naver.com/main/read.nhn?mode=LS2D...,"車,제조,현장,필수,장비,국산화,제품,특허,해외,공략,가속,한국,뉴매틱,공장,자동,...","車,제조,현장,필수,장비,국산화,車;제조,제조;현장,현장;필수,필수;장비,장비;국산...",16,0.0,0.245791
1,경제,증권,2019-01-01,아시아경제,긴장 넘쳤던 2018 증시…코스피 2년 만에 2100 하회,https://news.naver.com/main/read.nhn?mode=LS2D...,"지난해,국내,증시,긴장감,코스피,지수,대,대,내림,반복,코스닥,대,대,일,한국,거래...","지난해,국내,증시,국내;증시,긴장감,코스피,지수,코스피;지수,대,대,내림,반복,코스...",3,0.0,0.437175
2,경제,증권,2019-01-01,매일경제,"SK 주요 관계사 경영진, CES 2019 참가…성장동력 찾기 적극 나선다",https://news.naver.com/main/read.nhn?mode=LS2D...,"주요,관계사,오,일,일,미국,라스베가스,열리,세계,최대,가전,전시회,참가,새해,성장...","주요,관계사,주요;관계사,오,일,일,미국,라스베가스,미국;라스베가스,열리,세계,최대...",35,0.0,0.377346
3,경제,증권,2019-01-01,한국경제TV,"[신년대기획] 투자, 판이 바뀐다 1부 위기의 파고를 넘어",https://news.naver.com/main/read.nhn?mode=LS2D...,"동영상,뉴스,대한민국,정부,수립,오늘,이르,한강,기적,이,만큼,경제,발전,배경,금융...","동영상,뉴스,대한민국,정부,수립,동영상;뉴스,뉴스;대한민국,대한민국;정부,정부;수립...",44,0.0,0.144922
4,경제,증권,2019-01-01,조선비즈,증권사 리서치센터장이 뽑은 새해 유망주식 5개,https://news.naver.com/main/read.nhn?mode=LS2D...,"년,황금,돼지해,바라보,증권사,리서치,센터,장,마음,무겁,그간,홀로,호황,미국,증시...","년,황금,돼지해,년;황금,황금;돼지해,년;황금;돼지해,바라보,증권사,리서치,센터,장...",44,0.0,0.551857
5,경제,증권,2019-01-01,한국경제,"코스피, 2분기 '반등'…코스닥은 4차산업 등 성장주 유망",https://news.naver.com/main/read.nhn?mode=LS2D...,"올해,코스피,지수,미국,중앙은행,금리,인상,여파,상저,흐름,전망,이,코스닥,시장,차...","올해,코스피,지수,올해;코스피,코스피;지수,올해;코스피;지수,미국,중앙은행,미국;중...",44,0.0,0.318069
6,경제,증권,2019-01-01,한국경제,5대 증권사 리서치센터장들이 선정한 황금돼지해 투자 유망주는?,https://news.naver.com/main/read.nhn?mode=LS2D...,"년,증시,전망,텔레콤,상용,기대,배당주,매력,전기차,시장,확대,배터리,사,삼,성,주...","년,증시,전망,년;증시,증시;전망,년;증시;전망,텔레콤,상용,기대,배당주,매력,배당...",44,0.0,0.333955
7,경제,증권,2019-01-01,아이뉴스24,"불안한 새해 증시, ELS로 방어해볼까",https://news.naver.com/main/read.nhn?mode=LS2D...,"글로벌,증시,정치,이슈,등,외,요인,불안,모습,보이,있,때,증시,하락,어느,정도,방...","글로벌,증시,글로벌;증시,정치,이슈,등,외,이슈;등,등;외,이슈;등;외,요인,불안,...",43,0.0,0.725988
8,경제,증권,2019-01-01,아이뉴스24,"증권사들, 2019년 코스피 최고치는 '2400'",https://news.naver.com/main/read.nhn?mode=LS2D...,"국내,증권사,올해,코스피,지수,최고,정도,기록,것,이,평균,전망,그,동안,글로벌,경...","국내,증권사,국내;증권사,올해,코스피,지수,올해;코스피,코스피;지수,올해;코스피;지...",44,0.0,0.305958
9,경제,증권,2019-01-01,국민일보,올해 글로벌 증시는 ‘불확실성과의 싸움’,https://news.naver.com/main/read.nhn?mode=LS2D...,"올해,글로벌,증시,확실,치열,전투,것,관측,증권사,지난해,미,중,무역,전쟁,흥국,부...","올해,글로벌,증시,올해;글로벌,글로벌;증시,올해;글로벌;증시,확실,치열,전투,것,관...",44,0.0,0.292193


In [ ]:
# 어떤 토픽이 주식과 가장 연관있는 토픽인지 찾는 과정 진행
# 삼성전자의 주가와 거래랑에 영향을 미치는 토픽 선정

In [33]:
df_price[:10]

,open,high,low,close,volume
date,,,,,
2020-01-02,1555.0,1575.0,1545.0,1565.0,1407824.0
2020-01-03,1580.0,1585.0,1560.0,1565.0,1309077.0
2020-01-06,1550.0,1565.0,1500.0,1520.0,1457564.0
2020-01-07,1525.0,1560.0,1510.0,1545.0,1001700.0
2020-01-08,1525.0,1540.0,1415.0,1430.0,2952466.0
2020-01-09,1480.0,1535.0,1470.0,1480.0,1775063.0
2020-01-10,1485.0,1530.0,1480.0,1510.0,1122749.0
2020-01-13,1515.0,1595.0,1505.0,1560.0,3341499.0
2020-01-14,1565.0,1580.0,1535.0,1555.0,1050758.0


In [36]:
# 뉴스기사 데이터 프레임 (df)와 주식 데이터(df_price) 와의 상관관계 


n_topics = 50

df['date'] = pd.to_datetime(df['date'])
df[:10]

for i in range(n_topics):
  tmp = df[df.topic==i].groupby('date')['topic'].count()

  tmp2 = df_price[['volume']].corrwith(tmp) # corelation 계산

  break

In [37]:
tmp2

volume   -0.013508
dtype: float64